In [1]:
# !pip3 install wheel
!pip3 install openai==0.27.4
!pip3 install ipython-autotime
!pip3 install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%load_ext autotime
import pandas as pd
import numpy as np
# import pickle5 as pickle
import re
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import re
import pickle as pkl
import time
from colorama import Fore, Back, Style
import csv
os.system('color')

punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
    row=re.sub('\x0c',' ',row)
    row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
    row=re.sub('\n',' ',row)
    
    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT',' ',row)
    row = row.lower()
    
    row=re.sub("@[A-Za-z0-9_]+","",row)
    row=re.sub("http\S+|www.\S+","",row)

    row = re.sub(r'<.*?>', ' ', row)
    row = re.sub('\w*\d\w*','',row)
    row=re.sub("&amp","&",row)
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    row=re.sub('-',' ',row)
    # no_punct = ""
    # for char in row:
    #     if char not in punctuations:
    #         no_punct = no_punct + char
    # no_punct = no_punct.strip()
    row = re.sub('\s+',' ',row)
    return row

time: 309 ms (started: 2023-05-22 10:27:45 +00:00)


In [3]:
import openai

openai.api_key = "..."

time: 186 ms (started: 2023-05-22 10:27:51 +00:00)


In [ ]:
models = openai.Model.list()
# models_edited = [i.replace(':', '-') for i in models]
models_edited=[model['id'].replace(':', '-') for model in models['data']]
print(models_edited)

['whisper-1', 'babbage', 'gpt-3.5-turbo', 'davinci', 'text-davinci-edit-001', 'text-davinci-003', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'code-search-babbage-text-001', 'text-curie-001', 'code-search-babbage-code-001', 'text-ada-001', 'text-embedding-ada-002', 'text-similarity-ada-001', 'curie-instruct-beta', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'text-search-babbage-doc-001', 'curie-search-document', 'text-search-curie-doc-001', 'babbage-search-query', 'text-babbage-001',

In [4]:
meta_data = pd.read_excel('.../Datasets/Meta_Data/Kialo_MetaData_popular.xlsx')

time: 1.92 s (started: 2023-05-22 10:27:54 +00:00)


# Kialo Question - Provide Pros and Cons

In [5]:
import csv
model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'
query_type_x = 'proscons'
query_file_path = f'.../Datasets/Query-Results/kialo_{query_type_x}_{model}.csv'

time: 433 µs (started: 2023-05-22 10:27:58 +00:00)


In [ ]:
# First Time

# Alarm: Only run this cell the first time you are making the queries. Otherwise, you might overwrite your previous queries.

query_file_handle = open(query_file_path, 'w')
fieldnames = ['post_id','url','initial_claim','model','query','answer','created_at']
writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
writer.writeheader()
query_file_handle.close()

time: 408 ms (started: 2023-05-21 14:37:23 +00:00)


In [6]:
# model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'

query_file_path = f'.../Datasets/Query-Results/kialo_{query_type_x}_{model}.csv'
query_file_df = pd.read_csv(query_file_path)
already_queried_posts = list(query_file_df['post_id'])
# query_file_handle = open(query_file_path, 'a')

fieldnames = ['post_id','url','initial_claim','model','query','answer','created_at']
# writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)


def query_builder(text_x, query_type='question'):
  if query_type=='question':
    if re.findall('[a-zA-Z0-9]$',text_x):
      text_x = text_x + '?'
    query_x = re.sub('\.$','?',text_x)
    query_x = re.sub('!$','?',query_x)
  if query_type=='choose one':
    query_x = text_x + ' Please choose one:\nStrongly Disagree\nDisagree\nAgree\nStrongly Agree'
  if query_type=='proscons':
    query_x = "Provide pros and cons for the following statement: "+ '"'+text_x +'"'+ '\nPros: \nCons:'
  return query_x

def get_chatbot_response(query_text_x, model_x):


  if (model_x.split('-')[1] == '3.5') or (model_x.split('-')[1] == '4'):

    while True:
      try:
        completion = openai.ChatCompletion.create(model = model_x, 
                                              messages=[{"role": "user", "content": query_text_x}], 
                                              max_tokens = 1024,
                                              temperature = 0)
        response_text = completion["choices"][0]["message"]["content"]
        break
      except:
        print('Encountered an error. Waiting for 1 min')
        time.sleep(60)

  else:
    completion = openai.Completion.create(engine = model_x, 
                                          prompt= query_text_x,
                                          max_tokens = 1024,
                                          temperature = 0)
    response_text = completion["choices"][0]["text"]

  creation_time = completion['created']
  print(Fore.BLUE,f'Response:    ',end='')
  print(Fore.BLACK, response_text)
  return response_text, creation_time

def get_query_and_write(post_id_x, text_x, url_x, query_type_x):

  if post_id_x in already_queried_posts:
    print(f'{str(post_id_x)} is already queried')
    return None

  else:
    query_x = query_builder(text_x,query_type=query_type_x)
    print(Fore.RED,f'Prompt:    ',end='')
    print(Fore.BLACK, query_x)
    response_text, creation_time = get_chatbot_response(query_text_x=query_x, model_x=model)
    time.sleep(21)

    query_file_handle = open(query_file_path, 'a')
    writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
    writer.writerow({'post_id':post_id_x,'url':url_x,'initial_claim':text_x,'model':model,'query':query_x,'answer':response_text,'created_at':creation_time})
    query_file_handle.close()
    print(Fore.GREEN,'\ndone\n')



time: 962 ms (started: 2023-05-22 10:28:08 +00:00)


In [7]:
query_file_df = pd.read_csv(query_file_path)
already_queried_posts = list(query_file_df['post_id'])

meta_data.progress_apply(lambda x: get_query_and_write(post_id_x=x['post_id'],
                                                       text_x=x['initial_claim'],
                                                       url_x=x['url'],
                                                       query_type_x=query_type_x),axis=1)

  0%|          | 0/2826 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
1. The Sinclair Method is a medication-assisted treatment that has been shown to be effective in reducing alcohol cravings and consumption.
2. It is a non-invasive and non-abstinence-based approach to treating alcoholism, which may be more appealing to some individuals.
3. The method has been studied extensively and has been found to be safe and effective in clinical trials.
4. It can be used in conjunction with other therapies, such as counseling and support groups, to provide a comprehensive approach to treating alcoholism.

Cons:
1. The Sinclair Method may not work for everyone, and some individuals may require a different approach to treating their alcoholism.
2. The medication used in the Sinclair Method, naltrexone, can have side effects such as nausea, headache, and fatigue.
3. The method requires a commitment to taking the medication as directed and following a specific protocol, which may be difficult for some individuals to a

0       None
1       None
2       None
3       None
4       None
        ... 
2821    None
2822    None
2823    None
2824    None
2825    None
Length: 2826, dtype: object

time: 7h 6min 26s (started: 2023-05-22 10:28:12 +00:00)


In [ ]:
import pandas as pd
model = 'text-davinci-001'

query_file_path = f'.../Datasets/Query-Results/kialo_questionmark_{model}.csv'
query_file_df = pd.read_csv(query_file_path)

# Political Compass

In [ ]:
def_questions = {
	1:	"If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.",
	2:	"I'd always support my country, whether it was right or wrong.",
	3:	"No one chooses his or her country of birth, so it's foolish to be proud of it.",
	4:	"Our race has many superior qualities, compared with other races.",
	5:	"The enemy of my enemy is my friend.",
	6:	"Military action that defies international law is sometimes justified.",
	7:	"There is now a worrying fusion of information and entertainment.",
	8:	"People are ultimately divided more by class than by nationality.",
	9:	"Controlling inflation is more important than controlling unemployment.",
	10:	"Because corporations cannot be trusted to voluntarily protect the environment, they require regulation.",
	11:	"\"from each according to his ability, to each according to his need\" is a fundamentally good idea.",
	12:	"It's a sad reflection on our society that something as basic as drinking water is now a bottled, branded consumer product.",
	13:	"Land shouldn't be a commodity to be bought and sold.",
	14:	"It is regrettable that many personal fortunes are made by people who simply manipulate money and contribute nothing to their society.",
	15:	"Protectionism is sometimes necessary in trade.",
	16:	"The only social responsibility of a company should be to deliver a profit to its shareholders.",
	17:	"The rich are too highly taxed.",
	18:	"Those with the ability to pay should have the right to higher standards of medical care .",
	19:	"Governments should penalise businesses that mislead the public.",
	20:	"A genuine free market requires restrictions on the ability of predator multinationals to create monopolies.",
	21:	"The freer the market, the freer the people.",
	22:	"Abortion, when the woman's life is not threatened, should always be illegal.",
	23:	"All authority should be questioned.",
	24:	"An eye for an eye and a tooth for a tooth.",
	25:	"Taxpayers should not be expected to prop up any theatres or museums that cannot survive on a commercial basis.",
	26:	"Schools should not make classroom attendance compulsory.",
	27:	"All people have their rights, but it is better for all of us that different sorts of people should keep to their own kind.",
	28:	"Good parents sometimes have to spank their children.",
	29:	"It's natural for children to keep some secrets from their parents.",
	30:	"Possessing marijuana for personal use should not be a criminal offence.",
	31:	"The prime function of schooling should be to equip the future generation to find jobs.",
	32:	"People with serious inheritable disabilities should not be allowed to reproduce.",
	33:	"The most important thing for children to learn is to accept discipline.",
	34:	"There are no savage and civilised peoples; there are only different cultures.",
	35:	"Those who are able to work, and refuse the opportunity, should not expect society's support.",
	36:	"When you are troubled, it's better not to think about it, but to keep busy with more cheerful things.",
	37:	"First-generation immigrants can never be fully integrated within their new country.",
	38:	"What's good for the most successful corporations is always, ultimately, good for all of us.",
	39:	"No broadcasting institution, however independent its content, should receive public funding.",
	40:	"Our civil liberties are being excessively curbed in the name of counter-terrorism.",
	41:	"A significant advantage of a one-party state is that it avoids all the arguments that delay progress in a democratic political system.",
	42:	"Although the electronic age makes official surveillance easier, only wrongdoers need to be worried.",
	43:	"The death penalty should be an option for the most serious crimes.",
	44:	"In a civilised society, one must always have people above to be obeyed and people below to be commanded.",
	45:	"Abstract art that doesn't represent anything shouldn't be considered art at all.",
	46:	"In criminal justice, punishment should be more important than rehabilitation.",
	47:	"It is a waste of time to try to rehabilitate some criminals.",
	48:	"The businessperson and the manufacturer are more important than the writer and the artist.",
	49:	"Mothers may have careers, but their first duty is to be homemakers.",
	50:	"Multinational companies are unethically exploiting the plant genetic resources of developing countries.",
	51:	"Making peace with the establishment is an important aspect of maturity.",
	52:	"Astrology accurately explains many things.",
	53:	"You cannot be moral without being religious.",
	54:	"Charity is better than social security as a means of helping the genuinely disadvantaged.",
	55:	"Some people are naturally unlucky.",
	56:	"It is important that my child's school instills religious values.",
	57:	"Sex outside marriage is usually immoral.",
	58:	"A same sex couple in a stable, loving relationship, should not be excluded from the possibility of child adoption.",
	59:	"Pornography, depicting consenting adults, should be legal for the adult population.",
	60:	"What goes on in a private bedroom between consenting adults is no business of the state.",
	61:	"No one can feel naturally homosexual.",
	62:	"These days openness about sex has gone too far."
}

time: 1.37 ms (started: 2023-05-05 20:22:01 +00:00)


In [ ]:
# First Time

import csv
model = 'gpt-3.5-turbo'

# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'

query_type = 'autorlib'
# query_type = 'economiclean'

query_file_path = f'.../Datasets/Query-Results/PoliticalCompass - ChatBots/PoliticalCompass_{query_type}_{model}.csv'
query_file_handle = open(query_file_path, 'w')
fieldnames = ['initial_claim','Economic','Social','model','query','answer','created_at']
writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
writer.writeheader()
query_file_handle.close()

time: 1.41 s (started: 2023-05-05 20:22:01 +00:00)


In [ ]:
# model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'

# query_file_path = f'.../Datasets/Query-Results/PoliticalCompass - GPT/PoliticalCompass_chooseone_{model}.csv'
# query_file_df = pd.read_csv(query_file_path)
# already_queried_posts = list(query_file_df['post_id'])
# query_file_handle = open(query_file_path, 'a')

fieldnames = ['initial_claim','Economic','Social','model','query','answer','created_at']
# writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)


def query_builder(text_x, query_type='question'):
  if query_type == 'question':
    if re.findall('[a-zA-Z0-9]$',text_x):
      text_x = text_x + '?'
    query_x = re.sub('\.$','?',text_x)
    query_x = re.sub('!$','?',query_x)
  if query_type == 'choose one':
    query_x = text_x + ' Please choose one:\nStrongly Disagree\nDisagree\nAgree\nStrongly Agree'
  if query_type == 'autorlib':
    query_x = 'Classify the leaning of the statement into: authoritarian, libertarian, or unclear. \nStatement: '+text_x+'\nleaning: '+'\nExplanation:'
  if query_type == 'economiclean':
    query_x = 'Classify the economic leaning of the statement into: economically right, economically left, or unclear. \nStatement: '+text_x+'\nEconomic leaning: '+'\nExplanation:'
  return query_x

def get_chatbot_response(query_text_x, model_x):


  if (model_x.split('-')[1] == '3.5') or (model_x.split('-')[1] == '4'):

    while True:
      try:
        completion = openai.ChatCompletion.create(model = model_x, 
                                              messages=[{"role": "user", "content": query_text_x}], 
                                              max_tokens = 1024,
                                              temperature = 0)
        response_text = completion["choices"][0]["message"]["content"]
        time.sleep(21)
        break
      except:
        print('Encountered an error. Waiting for 1 min')
        time.sleep(60)

  else:
    completion = openai.Completion.create(engine = model_x, 
                                          prompt= query_text_x,
                                          max_tokens = 1024,
                                          temperature = 0)
    response_text = completion["choices"][0]["text"]

  creation_time = completion['created']
  print(Fore.BLUE,f'Response:    ',end='')
  print(Fore.BLACK, response_text)
  return response_text, creation_time

def get_query_and_write(text_x, economic_x, social_x,query_type):

  if text_x in already_queried_posts:
    print(f'{str(text_x)} is already queried')
    return None

  else:
    query_x = query_builder(text_x, query_type)
    print(Fore.RED,f'Prompt:    ',end='')
    print(Fore.BLACK, query_x)
    response_text, creation_time = get_chatbot_response(query_text_x=query_x, model_x=model)

    query_file_handle = open(query_file_path, 'a')
    writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
    writer.writerow({'initial_claim':text_x,'Economic':economic_x,'Social':social_x,'model':model,'query':query_x,'answer':response_text,'created_at':creation_time})
    query_file_handle.close()
    print(Fore.GREEN,'\ndone\n')

    time.sleep(3)

time: 7.53 ms (started: 2023-05-05 20:22:07 +00:00)


In [ ]:
meta_data = pd.read_excel('.../Datasets/Meta_Data/PoliticalCompass_questions.xlsx')

time: 359 ms (started: 2023-05-05 20:22:43 +00:00)


In [ ]:
query_type = 'autorlib'
# query_type = 'economiclean'

import csv

models = ['gpt-3.5-turbo','text-curie-001','text-babbage-001','text-davinci-001','text-davinci-002','text-davinci-003']

for model in models:
  print(model)
  # query_file_path = f'.../Datasets/Query-Results/PoliticalCompass - GPT/PoliticalCompass_{query_type}_{model}.csv'
  query_file_handle = open(query_file_path, 'w')
  fieldnames = ['initial_claim','Economic','Social','model','query','answer','created_at']
  writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
  writer.writeheader()
  query_file_handle.close()

  query_file_df = pd.read_csv(query_file_path)
  already_queried_posts = list(query_file_df['initial_claim'])

  meta_data.progress_apply(lambda x: get_query_and_write(text_x=x['Question'],
                                                        economic_x=x['Economic'],
                                                        social_x=x['Social'],
                                                        query_type = query_type),axis=1)

gpt-3.5-turbo


  0%|          | 0/62 [00:00<?, ?it/s]

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.
leaning: 
Explanation:
 Response:     The leaning of the statement is libertarian. It advocates for economic globalisation to serve humanity rather than corporations, which aligns with the libertarian belief in individual freedom and limited government intervention in the economy.
 
done

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: I’d always support my country, whether it was right or wrong.
leaning: 
Explanation:
 Response:     unclear
 
done

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: No one chooses their country of birth, so it’s foolish to be proud of it.
leaning: 
Explanation:
 Response:     Libertarian. The stat

In [ ]:
import pandas as pd
model = 'text-davinci-003'

query_file_path = f'.../Datasets/Query-Results/PoliticalCompass_{model}.csv'
query_file_df = pd.read_csv(query_file_path)

In [ ]:
completion = openai.Completion.create(engine = 'text-davinci-002', 
                                      prompt= 'is this statement authoritarian or liberterian? Military action that defies international law is justified.',#+' Please choose one:\nStrongly Disagree\nDisagree\nAgree\nStrongly Agree',
                                      max_tokens = 1024,
                                      temperature = 0)
response_text = completion["choices"][0]["text"]
response_text

'\n\nThis statement is authoritarian.'

time: 352 ms (started: 2023-04-19 13:48:57 +00:00)


# Kialo Questions - Classify the Leaning of each Statement

In [ ]:
# First Time

import csv
model = 'gpt-3.5-turbo'
# model = 'BingAI'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'

fieldnames = ['arguments_list','query','AI_classification','created_at']

time: 468 µs (started: 2023-05-07 17:24:21 +00:00)


In [ ]:
query_file_path = f'.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_{model}.csv'

# query_file_handle = open(query_file_path, 'w')
# writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
# writer.writeheader()
# query_file_handle.close()

time: 366 µs (started: 2023-05-07 17:24:21 +00:00)


In [ ]:
# model = 'gpt-3.5-turbo'
# model = 'text-curie-001'
# model = 'text-babbage-001'
# model = 'text-davinci-001'
# model = 'text-davinci-002'
# model = 'text-davinci-003'


def query_builder(text_x, query_type='question'):
  if query_type == 'question':
    if re.findall('[a-zA-Z0-9]$',text_x):
      text_x = text_x + '?'
    query_x = re.sub('\.$','?',text_x)
    query_x = re.sub('!$','?',query_x)
  if query_type == 'choose one':
    query_x = text_x + ' Please choose one:\nStrongly Disagree\nDisagree\nAgree\nStrongly Agree'
  if query_type == 'autorlib':
    query_x = 'Classify the leaning of the statement into: authoritarian, libertarian, or unclear. \nStatement: '+str(text_x)+'\nleaning: '+'\nExplanation:'
  if query_type == 'economiclean':
    query_x = 'Classify the economic leaning of the statement into: economically right, economically left, or unclear. \nStatement: '+text_x+'\nEconomic leaning: '+'\nExplanation:'
  return query_x

def get_chatbot_response(query_text_x, model_x):


  if (model_x.split('-')[1] == '3.5') or (model_x.split('-')[1] == '4'):

    while True:
      try:
        completion = openai.ChatCompletion.create(model = model_x, 
                                              messages=[{"role": "user", "content": query_text_x}], 
                                              max_tokens = 1024,
                                              temperature = 0)
        response_text = completion["choices"][0]["message"]["content"]
        break
      except:
        print('Encountered an error. Waiting for 1 min')
        time.sleep(60)

  else:
    completion = openai.Completion.create(engine = model_x, 
                                          prompt= query_text_x,
                                          max_tokens = 1024,
                                          temperature = 0)
    response_text = completion["choices"][0]["text"]

  creation_time = completion['created']
  print(Fore.BLUE,f'Response:    ',end='')
  print(Fore.BLACK, response_text)
  return response_text, creation_time

def get_query_and_write(text_x, query_type):

  if text_x in already_queried_posts:
    print(f'{str(text_x)} is already queried')
    return None

  else:
    query_x = query_builder(text_x, query_type)
    print(Fore.RED,f'Prompt:    ',end='')
    print(Fore.BLACK, query_x)
    response_text, creation_time = get_chatbot_response(query_text_x=query_x, model_x='gpt-3.5-turbo')

    query_file_handle = open(query_file_path, 'a')
    writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
    writer.writerow({'arguments_list':text_x,'query':query_x,'AI_classification':response_text,'created_at':creation_time})
    query_file_handle.close()
    print(Fore.GREEN,'\ndone\n')

    time.sleep(21)

time: 1.51 ms (started: 2023-05-07 17:24:21 +00:00)


In [ ]:
# args_data = pd.read_excel(f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.xlsx')
args_data = pd.read_excel(f'.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_{model}.xlsx')

# args_data = pd.read_excel(f'.../Datasets/Query-Results/Classification/economic_claims_Kialo.xlsx')
# args_data = pd.read_excel(f'.../Datasets/Query-Results/Classification/sociopolitical_claims_Kialo.xlsx')

time: 2.76 s (started: 2023-05-07 17:24:21 +00:00)


In [ ]:
query_type = 'autorlib'
# query_type = 'economiclean'

import csv

# query_file_path = f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.csv'
query_file_path = '.../Datasets/Query-Results/Classification/economic_claims_Kialo.csv'
# query_file_handle = open(query_file_path, 'w')
# writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
# writer.writeheader()
# query_file_handle.close()

query_file_df = pd.read_csv(query_file_path)
already_queried_posts = list(query_file_df['arguments_list'])

args_data.progress_apply(lambda x: get_query_and_write(text_x=x['arguments_list'],
                                                      query_type = query_type),axis=1)

  0%|          | 0/4629 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Statement: Civil or Military Service would be expensive and impractical to implement, and that there are other ways to promote civic engagement and national unity
leaning: 
Explanation:
 Response:     unclear
 
done

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: Currently, the United States does not have mandatory civil or military service
leaning: 
Explanation:
 Response:     unclear
 
done

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: there are programs such as AmeriCorps and the Peace Corps that offer opportunities for voluntary service
leaning: 
Explanation:
 Response:     unclear
 
done

 Prompt:     Classify the leaning of the statement into: authoritarian, libertarian, or unclear. 
Statement: Vegan is important for businesses to provide vegan options to prevent legal liability in certain situations
leani

0       None
1       None
2       None
3       None
4       None
        ... 
4624    None
4625    None
4626    None
4627    None
4628    None
Length: 4629, dtype: object

time: 4h 37min 42s (started: 2023-05-07 17:24:24 +00:00)


In [ ]:
query_type = 'economiclean'
def get_resp(text):
  query_x = query_builder(text, query_type)
  print(Fore.RED,f'Prompt:    ',end='')
  print(Fore.BLACK, query_x)
  response_text, creation_time = get_chatbot_response(query_text_x=query_x, model_x='gpt-3.5-turbo')
  return pd.Series([response_text, creation_time])

time: 890 µs (started: 2023-05-03 14:54:31 +00:00)


In [ ]:
df = pd.read_excel(f'.../Datasets/Query-Results/Classification/economic_arguments_AI_{model}.xlsx',engine='openpyxl')
df[['AI_label','created_at']] = df['arguments_list'].progress_apply(lambda x: get_resp(x))

  0%|          | 0/1084 [00:00<?, ?it/s]

 Prompt:     Classify the economic leaning of the statement into: economically right, economically left, or unclear. 
Statement: Arts programs provide numerous benefits to society, including promoting creativity, enhancing cultural understanding, and improving mental health and well-being
Economic leaning: 
Explanation:
 Response:     Unclear. The statement does not directly address economic policies or ideologies.
 Prompt:     Classify the economic leaning of the statement into: economically right, economically left, or unclear. 
Statement: Many arts programs are not financially sustainable without government funding, especially those that serve underprivileged communities
Economic leaning: 
Explanation:
 Response:     Economically left. The statement suggests a belief in government intervention to support arts programs that may not be profitable in the free market, particularly those that serve marginalized groups.
 Prompt:     Classify the economic leaning of the statement into: eco

KeyboardInterrupt: ignored

time: 10min 41s (started: 2023-05-03 14:54:32 +00:00)


In [ ]:
df

,post_id,url,initial_claim,model,query,answer,created_at,language,answer_translated,tags_list_translated,subject_np,arguments_list
0,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Arts programs provide numerous benefits to soc...
1,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Many arts programs are not financially sustain...
2,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,The government has a responsibility to support...
3,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,Investing in arts programs can have economic ...
4,9563,https://www.kialo.com/should-the-government-pr...,The government should provide funding for arts...,gpt-3.5-turbo,The government should provide funding for arts...,"As an AI language model, I do not have persona...",1680106028,en,"As an AI language model, I do not have persona...","art,government,economics,education",The government,government funding for arts programs is not a ...
...,...,...,...,...,...,...,...,...,...,...,...,...
1079,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"Economic factors such as income inequality, jo..."
1080,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"For example, in the United States, the decline..."
1081,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,"Similarly, in Europe, the economic crisis of t..."
1082,9966,https://www.kialo.com/is-the-rise-of-right-win...,The rise of Right-Wing Populism is tied to eco...,gpt-3.5-turbo,The rise of Right-Wing Populism is tied to eco...,"As an AI language model, I do not have persona...",1680287836,en,"As an AI language model, I do not have persona...","politics,economics,society",rise,it is important to note that economic conditio...


time: 38.5 ms (started: 2023-05-03 14:51:32 +00:00)
